# Initialization

In [1]:
import pandas as pd

In [27]:
df = pd.read_csv("D:\Project\Fashionism-Machine-Learning\dataset7.csv")

# Recommendation system

In [28]:
# =================
#      IMPORT
# =================
# Import
import os # Path managing
import json # Json file
import pickle
import re
import cv2 # Another image processing
import concurrent.futures # Do parallel computing
import random
import shutil
import pandas as pd # To create dataset
import numpy as np # image processing
import matplotlib.pyplot as plt # plot image
from PIL import Image # For image processing
from numpy.linalg import norm # Normalize
from tqdm import tqdm # Logging the time
import matplotlib.image as mpimg

# Tensorflow
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.layers import GlobalMaxPooling2D
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input
from sklearn.neighbors import NearestNeighbors

In [55]:
df = pd.read_csv("dataset8.csv")

In [56]:
# =====================
#     FUNCTIONS
# =====================

def recommend(features,feature_list):
    neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)

    distances, indices = neighbors.kneighbors([features])

    return indices

def extract_feature(img_path, model):
  img = cv2.imread(img_path)
  img = cv2.resize(img, (150,150))
  img = np.array(img)
  expand_img = np.expand_dims(img, axis=0)
  pre_img = preprocess_input(expand_img)
  result = model.predict(pre_img).flatten()
  normalized = result/norm(result)
  return normalized

In [57]:
# =================
# MODELS IMPORT
# =================

model_top_down = load_model("D:\Project\Fashionism-Machine-Learning\Models/model_top_down.h5")
model_extraction = load_model("D:\Project\Fashionism-Machine-Learning\Models/feature_extract_model.h5")

top_feature_list = np.array(pickle.load(open("D:\Project\Fashionism-Machine-Learning\Models/top_feature_extraction.pkl", "rb")))
bottom_feature_list = np.array(pickle.load(open("D:\Project\Fashionism-Machine-Learning\Models/bottom_feature_extraction.pkl", "rb")))

top_filenames = pickle.load(open('D:\Project\Fashionism-Machine-Learning\Models/top_directory.pkl', "rb"))
bottom_filenames = pickle.load(open("D:\Project\Fashionism-Machine-Learning\Models/bottom_directory.pkl", "rb"))

top_filenames_df = pickle.load(open('D:\Project\Fashionism-Machine-Learning\Models/top_directory_df.pkl', 'rb'))
bottom_filenames_df = pickle.load(open('D:\Project\Fashionism-Machine-Learning\Models/bottom_directory_df.pkl', 'rb'))

In [58]:
# Upload the images
# =================

# Sebagai contoh, dia langsung menerima inputtan image aja
image_path = "Uploads/test.jpg"

# Image Classification
# =====================

# Image preprocessing
image = Image.open(image_path)
image = image.resize((150, 150))
image = np.array(image) / 255.0
image = np.expand_dims(image, axis=0)

# Predict the category
prediction = model_top_down.predict(image)
predicted_class = 'top' if prediction[0][0] > 0.5 else 'bottom'

print(predicted_class)

1/1 [==============================] - 0s 180ms/step
top


In [64]:
# Image recommendation
# ====================

if predicted_class == 'top':
  features = extract_feature(image_path, model_extraction)
  indices = recommend(features, top_feature_list)

  # Recommendation file_path
  target_file = top_filenames[indices[0][0]]

  # Ambil setId dari rekomendasi
  filtered_df = df.loc[df['file_path'] == target_file]

  set_id = filtered_df["setId"]

  for set in set_id:
    set_id = set
    break
  
  # Teruskan setId ke dataframe bottom
  recommended_path = bottom_filenames_df[bottom_filenames_df['setId'] == set_id]["file_path"]

  for path in recommended_path:
    recommended_path = path
    break
  
  print(recommended_path)

  # Recommend the bottom
  features = extract_feature(recommended_path, model_extraction)
#   indices = recommend(features, bottom_feature_list)

#   # Print the recommendation
#   target_files = bottom_filenames[indices[0][:5]]

#   price_output = []

#   for file in target_files:
#       filtered_df = bottom_filenames_df.loc[bottom_filenames_df['file_path'] == file]
#       prices = filtered_df["price"].values
#       if len(prices) > 0:
#           price_output.append(prices[0])
#       else:
#           price_output.append('Unknown')

#   # Iterate over the target files and display the images with labels
#   for i, target_file in enumerate(target_files):
#       image_path = target_file
#       img = mpimg.imread(image_path)
#       plt.imshow(img)
#       plt.axis('off')
#       plt.title(f"Price: {price_output[i]}") # Ini harus diubah biar hasil iterasinya di masukin ke json, yg bakal dikirim ke android
#       plt.show()


# # If the input is bottom

# else:
#   features = extract_feature(image_path, model_extraction)
#   indices = recommend(features, bottom_feature_list)

#   # Recommendation file_path
#   target_file = bottom_filenames[indices[0][0]]

#   # Ambil setId dari rekomendasi
#   filtered_df = df.loc[df['file_path'] == target_file]

#   set_id = filtered_df["setId"]

#   for set in set_id:
#     set_id = set
#     break
  
#   # Teruskan setId ke dataframe top
#   recommended_path = top_filenames_df[top_filenames_df['setId'] == set_id]["file_path"]

#   for path in recommended_path:
#     recommended_path = path
#     break

#   # Recommend the top
#   features = extract_feature(recommended_path, model_extraction)
#   indices = recommend(features, top_feature_list)

#   # Print the recommendation
#   target_files = top_filenames[indices[0][:5]]

#   price_output = []

#   for file in target_files:
#       filtered_df = top_filenames_df.loc[top_filenames_df['file_path'] == file]
#       prices = filtered_df["price"].values
#       if len(prices) > 0:
#           price_output.append(prices[0])
#       else:
#           price_output.append('Unknown')

#   # Iterate over the target files and display the images with labels
#   for i, target_file in enumerate(target_files):
#       image_path = target_file  # Lokasi image rekomendasi ada di target_file
#       img = mpimg.imread(image_path)
#       plt.imshow(img)
#       plt.axis('off')
#       plt.title(f"Price: {price_output[i]}") # Harga nya ada di price_output[i]
#       plt.show()

1/1 [==============================] - 0s 173ms/step


Dataset/4955186_m.jpg


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [41]:
import cv2
image = cv2.imread('Uploads/test.jpg')
if image is not None:
    print("Image loaded successfully.")
else:
    print("Failed to load the image.")
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Image loaded successfully.


In [38]:
image = cv2.imread('Uploads/test.jpg')

In [40]:
image.shape

(150, 150, 3)